<a href="https://colab.research.google.com/github/wdconinc/practical-computing-for-scientists/blob/master/Lectures/lecture10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lecture #10

##Standard Preamble

In [0]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math

##In our last episode

* Simple error propagation
* Plotting data with error bars.
* Fitting functions to data. The method of least squares and $\chi^2$
* Numerical derivatives. Accuracy: truncation error and roundoff error.

##Derivatives

###Defining and testing D(f,x)

In [0]:
def D(f, x, h = 1.0e-6):
    ''' computes the derivative df/dx '''
    return (f(x+h) - f(x-h)) / (2.0*h)

def Dimproved(f, x, h = 1.0e-6):
    ''' computes the derivative df/dx '''
    temp = x+h
    h = temp-x
    return (f(x+h) - f(x-h)) / (2.0*h)

f = lambda z : z + z**2
df = lambda z : 1 + 2*z
x = np.linspace(-3,3)
plt.plot(x,f(x))
plt.plot(x,df(x))
plt.plot(x,D(f,x),'--r')

###D(f,x) - df(x) and the choice of h

In [0]:
plt.plot(x,D(f, x, h = 1.0e-6) - df(x))

###Making it more quantitative - rms vs exponent

In [0]:
std_dev = []
std_dev2 = []
exponents = range(2,10)
for n in exponents:
    rms = np.std(D(f, x, h = 10.0**(-n)) - df(x))
    std_dev.append(rms)
    rms = np.std(Dimproved(f, x, h = 10.0**(-n)) - df(x))
    std_dev2.append(rms)

plt.plot(exponents, std_dev)
plt.plot(exponents, std_dev2, '--r')

###Derivative of the sin() function

In [0]:
std_dev = []
std_dev2 = []
exponents = range(5,7)
for n in exponents:
    rms = np.std(D(np.sin, x, h = 10.0**(-n)) - np.cos(x))
    std_dev.append(rms)
    rms = np.std(Dimproved(np.sin, x, h = 10.0**(-n)) - np.cos(x))
    std_dev2.append(rms)

plt.plot(exponents,std_dev)
plt.plot(exponents,std_dev2,'--r')

### What's going on here?

The error has two components, the _truncation error_:

$$e_t = \left | \frac{h^2 f'''(x)}{6} \right |$$ 

due to the lack of higher order terms in 

$$ \frac{\mathrm{d}f}{\mathrm{d}x} = \frac{f(x+h) - f(x-h)}{2h} $$
 
and the _round-off error_: 

$$e_r \approx \epsilon_f \left | \frac{f(x)}{h} \right | $$

due to round-off uncertainties in the computation of $f(x)$. The magnitude of these uncertainties as a fraction of $f(x)$, written here as $\epsilon_f$, will depend on the function and how complicated it is to compute it.   Picking a small $h$ blows up $e_r$ whereas picking a large $h$ blows up $e_t$ (unless $f''' =0 $ as with a quadratic function).

### The best choice of h?

We can find a best choice of h by solving

$$\frac{\partial}{\partial h} (e_t+e_r) = 0$$

for h. The solution is 

$$h = \left (3 \epsilon_f \frac{f}{f'''} \right )^{1/3}$$

Considering the practical case $f(x)=\sin{x}$,  $f/f'''$ is usually around 1, except when $f'''$ is very small, in which case even a large $h$ would do.  The machine precision for a 64-bit floating point number is $\epsilon_m \approx 10^{-16}$ and it's not too hard to imagine it takes somewhere between $N \approx 1 \ldots 100$ operations to compute $\sin{x}$. Thus

$$h \approx \left ( N \epsilon_m \right )^{1/3}$$

which ranges from $6\times 10^{-6}$ to $3 \times 10^{-5}$, in pretty good agreement with our plot.

### The magnitude of $e_t + e_r$

If we plug our optimal choice of $h$ back into the formula for $e_t + e_r$ we can find that the fractional error is

$$\frac{e_t + e_r}{f'} \approx \epsilon_f^{2/3}$$

For $\epsilon_f = N \epsilon_m$ this gives an uncertainty between $4\times 10^{-11}$ and $8 \times 10^{-10}$ which also appears to agree with the plot above. Note, the plot shows something more like $e_t + e_r$ but $|f'(x)|\leq1$ for $f(x)=\sin{x}$ so the fractional uncertainty has the same magnitude.



###One more trick, involving h

See $Dimproved(f,x,h)$ above. The trick `temp = x + h` followed by `h = temp - x` makes sure that `h` is a perfectly represented number, rounded to the precision of `x` in both the numerator and denominator. If we don’t do this we incur a fractional error $\delta h/h = \epsilon_m x/h$.


##Root Finding

###Plot the Function

In [0]:
g = lambda x: np.cos(8*x) + x**2 - x
x = np.linspace(-1, 1.5, 300)
plt.plot(x,g(x))
plt.axhline(color = 'red')

###An algorithm to bracket the roots

In [0]:
def bracket_roots(f, xlow, xhigh, n = 100):
    result = []
    dx = (xhigh - xlow) / float(n)
    for i in range(0,n):
        a = xlow + dx*i
        b = a + dx
        if f(a)*f(b) < 0.0:
            result.append((a,b))
    return result

brackets = bracket_roots(D(g), -1.0, 1.5, n = 100)
print(brackets)

###Root polishing - Bisection method

In [0]:
def bisection(f, xlow, xhigh, accuracy = 1e-6, nmax = 100):
    if xlow > xhigh:
        raise RuntimeError('xlow > xhigh')
    flow = f(xlow)
    fhigh = f(xhigh)
    for i in range(0,nmax):
        xmid = (xlow + xhigh) / 2.0
        fmid = f(xmid)
        if fmid == 0 or math.fabs(xhigh - xlow) < accuracy:
            return xmid
        if fmid*fhigh < 0:
            xlow, flow = xmid, fmid
        else:
            xhigh, fhigh = xmid, fmid
    
for xlow, xhigh in brackets:
    root = bisection(D(g), xlow, xhigh)
    print("found root %f between [%f,%f]" % (root, xlow, xhigh))

### Exercise: What are the maxima of the function g(x)

Reminder: the maxima are those points where the derivative is zero and the second derivative is positive.

In [0]:
# A more clever derivative which we can reuse
def D(f, h = 1.0e-6):
  return lambda x: (f(x+h) - f(x-h)) / (2*h)

g = lambda x: np.cos(8*x) + x**2 - x
x = np.linspace(-1, 1.5, 300)
plt.plot(x,g(x), color = "blue")
plt.plot(x,D(g)(x), color = "green")
plt.axhline(color = 'red')

In [0]:
for xlow, xhigh in bracket_roots(D(g), -1.0, 1.5, n = 100):
    root = bisection(D(g), xlow, xhigh)
    if D(D(g))(root) < 0:
      print("found maximum %f between [%f,%f] with 2nd derivative %f" % (root, xlow, xhigh, D(D(g))(root)))
    else:
      print("found minimum %f between [%f,%f] with 2nd derivative %f" % (root, xlow, xhigh, D(D(g))(root)))

###The secant method

###Brent's method (via scipy)

###Newton-Raphson Method